#### 1. Import pandas library

In [1]:
import pandas as pd
import numpy as np

#### 2. Import users table:

In [2]:
users = pd.read_csv("users_table.csv")
users.head()

,Id,Reputation,CreationDate,DisplayName,LastAccessDate,WebsiteUrl,Location,AboutMe,Views,UpVotes,DownVotes,AccountId,Age,ProfileImageUrl
0,-1,1,2010-07-19 06:55:26,Community,2010-07-19 06:55:26,http://meta.stackexchange.com/,on the server farm,"<p>Hi, I'm not really a person.</p>\n\n<p>I'm ...",0,5007,1920,-1,NaN,NaN
1,2,101,2010-07-19 14:01:36,Geoff Dalgas,2013-11-12 22:07:23,http://stackoverflow.com,"Corvallis, OR",<p>Developer on the StackOverflow team. Find ...,25,3,0,2,37.0,NaN
2,3,101,2010-07-19 15:34:50,Jarrod Dixon,2014-08-08 06:42:58,http://stackoverflow.com,"New York, NY","<p><a href=""http://blog.stackoverflow.com/2009...",22,19,0,3,35.0,NaN
3,4,101,2010-07-19 19:03:27,Emmett,2014-01-02 09:31:02,http://minesweeperonline.com,"San Francisco, CA",<p>currently at a startup in SF</p>\n\n<p>form...,11,0,0,1998,28.0,http://i.stack.imgur.com/d1oHX.jpg
4,5,6792,2010-07-19 19:03:57,Shane,2014-08-13 00:23:47,http://www.statalgo.com,"New York, NY",<p>Quantitative researcher focusing on statist...,1145,662,5,54503,35.0,NaN


#### 3. Rename Id column to userId

In [3]:
users = users.rename(columns = {"Id":"userId"})
users.head()

,userId,Reputation,CreationDate,DisplayName,LastAccessDate,WebsiteUrl,Location,AboutMe,Views,UpVotes,DownVotes,AccountId,Age,ProfileImageUrl
0,-1,1,2010-07-19 06:55:26,Community,2010-07-19 06:55:26,http://meta.stackexchange.com/,on the server farm,"<p>Hi, I'm not really a person.</p>\n\n<p>I'm ...",0,5007,1920,-1,NaN,NaN
1,2,101,2010-07-19 14:01:36,Geoff Dalgas,2013-11-12 22:07:23,http://stackoverflow.com,"Corvallis, OR",<p>Developer on the StackOverflow team. Find ...,25,3,0,2,37.0,NaN
2,3,101,2010-07-19 15:34:50,Jarrod Dixon,2014-08-08 06:42:58,http://stackoverflow.com,"New York, NY","<p><a href=""http://blog.stackoverflow.com/2009...",22,19,0,3,35.0,NaN
3,4,101,2010-07-19 19:03:27,Emmett,2014-01-02 09:31:02,http://minesweeperonline.com,"San Francisco, CA",<p>currently at a startup in SF</p>\n\n<p>form...,11,0,0,1998,28.0,http://i.stack.imgur.com/d1oHX.jpg
4,5,6792,2010-07-19 19:03:57,Shane,2014-08-13 00:23:47,http://www.statalgo.com,"New York, NY",<p>Quantitative researcher focusing on statist...,1145,662,5,54503,35.0,NaN


#### 4. Import posts table:

In [4]:
posts = pd.read_csv("posts_table.csv")
posts.head()

,Id,PostTypeId,AcceptedAnswerId,CreaionDate,Score,ViewCount,Body,OwnerUserId,LasActivityDate,Title,...,AnswerCount,CommentCount,FavoriteCount,LastEditorUserId,LastEditDate,CommunityOwnedDate,ParentId,ClosedDate,OwnerDisplayName,LastEditorDisplayName
0,1,1,15.0,2010-07-19 19:12:12,23,1278.0,<p>How should I elicit prior distributions fro...,8.0,2010-09-15 21:08:26,Eliciting priors from experts,...,5.0,1,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1,59.0,2010-07-19 19:12:57,22,8198.0,<p>In many different statistical methods there...,24.0,2012-11-12 09:21:54,What is normality?,...,7.0,1,8.0,88.0,2010-08-07 17:56:44,NaN,NaN,NaN,NaN,NaN
2,3,1,5.0,2010-07-19 19:13:28,54,3613.0,<p>What are some valuable Statistical Analysis...,18.0,2013-05-27 14:48:36,What are some valuable Statistical Analysis op...,...,19.0,4,36.0,183.0,2011-02-12 05:50:03,2010-07-19 19:13:28,NaN,NaN,NaN,NaN
3,4,1,135.0,2010-07-19 19:13:31,13,5224.0,<p>I have two groups of data. Each with a dif...,23.0,2010-09-08 03:00:19,Assessing the significance of differences in d...,...,5.0,2,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,2,NaN,2010-07-19 19:14:43,81,NaN,"<p>The R-project</p>\n\n<p><a href=""http://www...",23.0,2010-07-19 19:21:15,NaN,...,NaN,3,NaN,23.0,2010-07-19 19:21:15,2010-07-19 19:14:43,3.0,NaN,NaN,NaN


#### 5. Rename Id column to postId and OwnerUserId to userId

In [5]:
posts = posts.rename(columns = {"Id":"postId", "OwnerUserId":"userId"})
posts.head()


,postId,PostTypeId,AcceptedAnswerId,CreaionDate,Score,ViewCount,Body,userId,LasActivityDate,Title,...,AnswerCount,CommentCount,FavoriteCount,LastEditorUserId,LastEditDate,CommunityOwnedDate,ParentId,ClosedDate,OwnerDisplayName,LastEditorDisplayName
0,1,1,15.0,2010-07-19 19:12:12,23,1278.0,<p>How should I elicit prior distributions fro...,8.0,2010-09-15 21:08:26,Eliciting priors from experts,...,5.0,1,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1,59.0,2010-07-19 19:12:57,22,8198.0,<p>In many different statistical methods there...,24.0,2012-11-12 09:21:54,What is normality?,...,7.0,1,8.0,88.0,2010-08-07 17:56:44,NaN,NaN,NaN,NaN,NaN
2,3,1,5.0,2010-07-19 19:13:28,54,3613.0,<p>What are some valuable Statistical Analysis...,18.0,2013-05-27 14:48:36,What are some valuable Statistical Analysis op...,...,19.0,4,36.0,183.0,2011-02-12 05:50:03,2010-07-19 19:13:28,NaN,NaN,NaN,NaN
3,4,1,135.0,2010-07-19 19:13:31,13,5224.0,<p>I have two groups of data. Each with a dif...,23.0,2010-09-08 03:00:19,Assessing the significance of differences in d...,...,5.0,2,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,2,NaN,2010-07-19 19:14:43,81,NaN,"<p>The R-project</p>\n\n<p><a href=""http://www...",23.0,2010-07-19 19:21:15,NaN,...,NaN,3,NaN,23.0,2010-07-19 19:21:15,2010-07-19 19:14:43,3.0,NaN,NaN,NaN


#### 6. Define new dataframes for users and posts with the following selected columns:
    **users columns**: userId, Reputation,Views,UpVotes,DownVotes
    **posts columns**: postId, Score,userId,ViewCount,CommentCount

In [6]:
users2 = users[["userId","Reputation", "Views", "UpVotes", "DownVotes"]]
posts2 = posts[["postId","Score","userId","ViewCount","CommentCount"]]

#### 7. Merge both dataframes, users and posts. 
You will need to make a [merge](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) of posts and users dataframes.

In [7]:
users2.head()

,userId,Reputation,Views,UpVotes,DownVotes
0,-1,1,0,5007,1920
1,2,101,25,3,0
2,3,101,22,19,0
3,4,101,11,0,0
4,5,6792,1145,662,5


In [8]:
posts2.head()

,postId,Score,userId,ViewCount,CommentCount
0,1,23,8.0,1278.0,1
1,2,22,24.0,8198.0,1
2,3,54,18.0,3613.0,4
3,4,13,23.0,5224.0,2
4,5,81,23.0,NaN,3


In [9]:
#used the index from both dataframes as key
merged = pd.merge(users2,posts2, left_index = True, right_index = True)
merged.head()

,userId_x,Reputation,Views,UpVotes,DownVotes,postId,Score,userId_y,ViewCount,CommentCount
0,-1,1,0,5007,1920,1,23,8.0,1278.0,1
1,2,101,25,3,0,2,22,24.0,8198.0,1
2,3,101,22,19,0,3,54,18.0,3613.0,4
3,4,101,11,0,0,4,13,23.0,5224.0,2
4,5,6792,1145,662,5,5,81,23.0,NaN,3


#### 8. How many missing values do you have in your merged dataframe? On which columns?

In [10]:
merged.isnull().sum()

userId_x            0
Reputation          0
Views               0
UpVotes             0
DownVotes           0
postId              0
Score               0
userId_y         1038
ViewCount       24105
CommentCount        0
dtype: int64

In [13]:
merged[merged.userId_y.isnull()].head()

,userId_x,Reputation,Views,UpVotes,DownVotes,postId,Score,userId_y,ViewCount,CommentCount
16,19,101,13,2,0,17,9,NaN,1261.0,0
27,34,185,27,23,0,28,6,NaN,NaN,0
54,67,101,4,0,0,56,56,NaN,NaN,7
94,115,101,8,0,0,101,40,NaN,NaN,6
142,169,113,19,5,0,152,9,NaN,1008.0,2


In [16]:
merged[merged.ViewCount.isnull()].head()

,userId_x,Reputation,Views,UpVotes,DownVotes,postId,Score,userId_y,ViewCount,CommentCount
4,5,6792,1145,662,5,5,81,23.0,NaN,3
8,10,121,20,2,0,9,13,50.0,NaN,3
11,13,817,178,44,1,12,20,5.0,NaN,1
12,15,11,8,0,0,13,14,23.0,NaN,4
13,16,101,4,7,0,14,5,36.0,NaN,1


#### 9. You will need to make something with missing values.  Will you clean or filling them? Explain. 
**Remember** to check the results of your code before passing to the next step

In [11]:
# The ViewCount series has over 50% of Nan Values, I believe the best approach is to remove this column, 
# as no analysis will be accurate. 
# Then I will drop userID_y because they are less than 10% of the total

In [18]:
# how many values would be dropped if dropna was applied - almost half
merged.dropna(how="any").shape

(15390, 10)

In [19]:
merged.shape

(40000, 10)

In [20]:
merged.dropna(how="all").shape # doesnt make any changes. There are no rows with all missing values

(40000, 10)

In [22]:
merged.dropna(subset = ["userId_y"], how = "any").shape #how shape would look like after droping user_id_y rows

(38962, 10)

In [25]:
merged = merged.dropna(subset = ["userId_y"], how = "any")

In [27]:
# how many ViewCount Nan Values ?
merged["ViewCount"].value_counts(dropna=False)

NaN       23572
98.0         43
150.0        43
122.0        42
156.0        41
          ...  
1733.0        1
8688.0        1
8108.0        1
3400.0        1
7687.0        1
Name: ViewCount, Length: 3403, dtype: int64

In [28]:
# remove ViewCount column
del merged["ViewCount"]

In [29]:
merged.columns

Index(['userId_x', 'Reputation', 'Views', 'UpVotes', 'DownVotes', 'postId',
       'Score', 'userId_y', 'CommentCount'],
      dtype='object')

#### 10. Adjust the data types in order to avoid future issues. Which ones should be changed? 

In [30]:
merged.dtypes

userId_x          int64
Reputation        int64
Views             int64
UpVotes           int64
DownVotes         int64
postId            int64
Score             int64
userId_y        float64
CommentCount      int64
dtype: object

In [33]:
# change userId_y to int
merged = merged.astype("int")
merged.dtypes

userId_x        int64
Reputation      int64
Views           int64
UpVotes         int64
DownVotes       int64
postId          int64
Score           int64
userId_y        int64
CommentCount    int64
dtype: object